# Router

> Convenience router factory that wires up standard card stack routes (Tier 2 API).

In [ ]:
#| default_exp routes.router

In [ ]:
#| export
from typing import Any, Callable, List, Optional, Tuple

from fasthtml.common import APIRouter

from cjm_fasthtml_card_stack.core.config import CardStackConfig
from cjm_fasthtml_card_stack.core.html_ids import CardStackHtmlIds
from cjm_fasthtml_card_stack.core.models import CardStackState, CardStackUrls
from cjm_fasthtml_card_stack.routes.handlers import (
    card_stack_navigate,
    card_stack_navigate_to_index,
    card_stack_update_viewport,
    card_stack_save_width,
    card_stack_save_scale,
)

## init_card_stack_router

Factory function that creates an `APIRouter` with all standard card stack routes
wired up. Returns a tuple of `(APIRouter, CardStackUrls)` so the consumer has
access to the route URLs for keyboard navigation and component rendering.

For consumers who need custom before/after logic in their handlers
(e.g., resetting a caret position before entering split mode), use the
Tier 1 response builder functions from `routes.handlers` directly instead.

In [ ]:
#| export
def init_card_stack_router(
    config: CardStackConfig,  # Card stack configuration
    state_getter: Callable[[], CardStackState],  # Function to get current state
    state_setter: Callable[[CardStackState], None],  # Function to save state
    get_items: Callable[[], List[Any]],  # Function to get current items list
    render_card: Callable,  # Card renderer callback: (item, CardRenderContext) -> FT
    route_prefix: str = "/card-stack",  # Route prefix for all card stack routes
    progress_label: str = "Item",  # Label for progress indicator
) -> Tuple[APIRouter, CardStackUrls]:  # (router, urls) tuple
    """Initialize an APIRouter with all standard card stack routes."""
    router = APIRouter(prefix=route_prefix)
    ids = CardStackHtmlIds(prefix=config.prefix)

    # -----------------------------------------------------------------
    # Navigation Routes
    # -----------------------------------------------------------------

    def _nav(direction: str) -> Any:
        """Shared navigation handler."""
        state = state_getter()
        items = get_items()
        result = card_stack_navigate(
            direction=direction, card_items=items, state=state,
            config=config, ids=ids, urls=urls,
            render_card=render_card, progress_label=progress_label,
        )
        state_setter(state)
        return result

    @router
    def nav_up() -> Any:
        """Navigate to previous item."""
        return _nav("up")

    @router
    def nav_down() -> Any:
        """Navigate to next item."""
        return _nav("down")

    @router
    def nav_first() -> Any:
        """Navigate to first item."""
        return _nav("first")

    @router
    def nav_last() -> Any:
        """Navigate to last item."""
        return _nav("last")

    @router
    def nav_page_up() -> Any:
        """Navigate up by page."""
        return _nav("page_up")

    @router
    def nav_page_down() -> Any:
        """Navigate down by page."""
        return _nav("page_down")

    @router
    def nav_to_index(target_index: int) -> Any:
        """Navigate to a specific item index (click-to-focus)."""
        state = state_getter()
        items = get_items()
        result = card_stack_navigate_to_index(
            target_index=target_index, card_items=items, state=state,
            config=config, ids=ids, urls=urls,
            render_card=render_card, progress_label=progress_label,
        )
        state_setter(state)
        return result

    # -----------------------------------------------------------------
    # Viewport Route
    # -----------------------------------------------------------------

    @router
    def update_viewport(visible_count: int) -> Any:
        """Update viewport with new card count (full outerHTML swap)."""
        state = state_getter()
        items = get_items()
        result = card_stack_update_viewport(
            visible_count=visible_count, card_items=items, state=state,
            config=config, ids=ids, urls=urls, render_card=render_card,
        )
        state_setter(state)
        return result

    # -----------------------------------------------------------------
    # Preference Persistence Routes
    # -----------------------------------------------------------------

    @router
    def save_width(card_width: int) -> None:
        """Save card stack width to server state."""
        state = state_getter()
        card_stack_save_width(state, card_width, config)
        state_setter(state)

    @router
    def save_scale(card_scale: int) -> None:
        """Save card stack scale to server state."""
        state = state_getter()
        card_stack_save_scale(state, card_scale, config)
        state_setter(state)

    # -----------------------------------------------------------------
    # Build URL bundle from registered routes
    # -----------------------------------------------------------------

    urls = CardStackUrls(
        nav_up=nav_up.to(),
        nav_down=nav_down.to(),
        nav_first=nav_first.to(),
        nav_last=nav_last.to(),
        nav_page_up=nav_page_up.to(),
        nav_page_down=nav_page_down.to(),
        nav_to_index=nav_to_index.to(),
        update_viewport=update_viewport.to(),
        save_width=save_width.to(),
        save_scale=save_scale.to(),
    )

    return router, urls

## Tests

In [ ]:
from cjm_fasthtml_card_stack.core.config import _reset_prefix_counter
from cjm_fasthtml_card_stack.core.models import CardRenderContext
from fasthtml.common import Div, Span

# Simple render_card for testing
def _test_render(item, ctx: CardRenderContext):
    return Div(Span(f"{item}"), cls=f"card-{ctx.card_role}")

# Simple state storage
_reset_prefix_counter()
_state = CardStackState()
_items = [f"Test item {i}" for i in range(10)]

def _get_state(): return _state
def _set_state(s): global _state; _state = s
def _get_items(): return _items

print("Test setup ready.")

Test setup ready.


In [ ]:
# Test router creation and URL generation
_reset_prefix_counter()
config = CardStackConfig(prefix="demo")

router, urls = init_card_stack_router(
    config=config,
    state_getter=_get_state,
    state_setter=_set_state,
    get_items=_get_items,
    render_card=_test_render,
    route_prefix="/cs",
)

assert router is not None
assert router.prefix == "/cs"
print(f"Router prefix: {router.prefix}")

Router prefix: /cs


In [ ]:
# Test URL generation
assert urls.nav_up == "/cs/nav_up"
assert urls.nav_down == "/cs/nav_down"
assert urls.nav_first == "/cs/nav_first"
assert urls.nav_last == "/cs/nav_last"
assert urls.nav_page_up == "/cs/nav_page_up"
assert urls.nav_page_down == "/cs/nav_page_down"
assert urls.nav_to_index == "/cs/nav_to_index"
assert urls.update_viewport == "/cs/update_viewport"
assert urls.save_width == "/cs/save_width"
assert urls.save_scale == "/cs/save_scale"
print("All URL generation tests passed!")
print(f"Sample URL: {urls.nav_up}")

All URL generation tests passed!
Sample URL: /cs/nav_up


In [ ]:
# Test multi-instance routers produce unique URL prefixes
config_a = CardStackConfig(prefix="text")
config_b = CardStackConfig(prefix="vad")

router_a, urls_a = init_card_stack_router(
    config_a, _get_state, _set_state, _get_items, _test_render,
    route_prefix="/text-stack",
)
router_b, urls_b = init_card_stack_router(
    config_b, _get_state, _set_state, _get_items, _test_render,
    route_prefix="/vad-stack",
)

assert urls_a.nav_up == "/text-stack/nav_up"
assert urls_b.nav_up == "/vad-stack/nav_up"
assert urls_a.nav_up != urls_b.nav_up
print("Multi-instance URL uniqueness tests passed!")

Multi-instance URL uniqueness tests passed!


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()